## Exemple using MLP multi-class with boosting and majority vote

# loading Dataset Iris

In [1]:
import time
import gc
import os
from contextlib import contextmanager
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print('[{0}] done in {1}'.format(name,time.time() - t0))
    

with timer('Loading Data'):
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

[Loading Data] done in 0.00187301635742


In [2]:
#transform dataset in dataframe

df_data = pd.DataFrame(data=X,columns=iris.feature_names)
df_trgt = pd.DataFrame(data=y,columns=['target'])
df = pd.concat([df_data,df_trgt],axis=1)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
df_data.shape

(150, 4)

# creating estimators

In [4]:
#from sklearn.neural_network import MLPClassifier
from Functions.mlpClassification import MLPKeras
from Functions.preprocessing import CrossValidation,CVEnsemble
from Functions.ensemble import AdaBoost
#from hep_ml.nnet import MLPClassifier
from hep_ml.nnet import MLPMultiClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split


mlp = MLPKeras(hidden_layer_sizes=(100,),
                 activation=('tanh','softmax'),
                 optimize='adam',
                 loss='mean_squared_error',
                 n_init=2,
                 batch_size=None,
                 epoch=200,
                 shuffle=True,
                 random_state=None,
                 verbose=0,
                 train_log=True,
                 early_stopping = False,
                 save_best_model = True,
                 monitor='acc',
                 metrics=['acc'],
                 validation_fraction=0.0,
                 dir='./')

#cv = CrossValidation(mlp,df_data.values,df_trgt.values)



boost = AdaBoost(base_estimator=mlp,
                           n_estimators=5,
                           learning_rate=1,
                           algorithm= 'SAMME',
                           random_state=None)

cv = CVEnsemble(boost,df_data.values,df_trgt.values,n_folds=3)

Using TensorFlow backend.


In [5]:
cv.fit(n_jobs=5)

In [13]:
mlp.get_params().keys()[-1]

'dir'

In [5]:
cv.predict_ifold(0)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2])

# Pre-processing of dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# fit Ensemble

In [6]:
#Usingo only MLP simple 
#mlp.fit(data_preproc_train,y_train)

#using MLP with boosting
boost.fit(X_train,y_train)

[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits
[+] 1 of 1 inits


AdaBoost(algorithm='SAMME.R',
     base_estimator=MLPSKlearn(activation=None, alpha=None, batch_size=None, beta_1=None,
      beta_2=None, dir=None, early_stopping=None, epsilon=None,
      hidden_layer_sizes=None, learning_rate=None, learning_rate_init=None,
      max_iter=None, momentum=None, n_iter_no_change=None,
      nesterovs_momentum=None, power_t=None, shuffle=None, solver=None,
      tol=None, validation_fraction=None, verbose=None, warm_start=None),
     learning_rate=1.0, n_estimators=20, random_state=None)

In [16]:
boost.set_classes(y_test)
output = boost.predic_maj(X_test)

/home/venancio/sonarteste/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
from sklearn.metrics import classification_report,confusion_matrix

print classification_report(y_pred=output,y_true=y_test)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         8
          1       0.27      0.43      0.33         7
          2       0.79      1.00      0.88        15

avg / total       0.46      0.60      0.52        30

